In [95]:
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
from collections import Counter
import xgboost as xgb
%matplotlib inline

Get freelancer profile by key

Endpoint
GET /api/profiles/v1/providers/{profile_key}.{format}

In [96]:
def json_prep(in_file):
    # read the entire file `ainto a python array
    with open(in_file, 'rb') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)

    # each element of 'data' is an individual JSON object.
    # i want to convert it into an *array* of JSON objects
    # which, in and of itself, is one large JSON object
    # basically... add square brackets to the beginning
    # and end, and have all the individual business JSON objects
    # separated by a comma
    data_json_str = "[" + ','.join(data) + "]"

    data_list_of_dicts = json.loads(data_json_str)
    
    out_df = pd.read_json(data_json_str)
    return out_df

## Make pandas DataFrame out of detailed profiles data for exploration

In [97]:
as_profiles_df = json_prep('../../data/detailed_profiles_da_0_8715.txt')

In [91]:
as_profiles_df.T.head(48)

,0,1,2,3,4,5,6,7,8,9,...,8704,8705,8706,8707,8708,8709,8710,8711,8712,8713
ag_cny_recno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ag_country,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ag_country_tz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ag_description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ag_logo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ag_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ag_recent_hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ag_total_hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
agency_ciphertext,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
assignments,"{u'hr': u'', u'fp': {u'job': {u'as_to': u'01/2...","{u'hr': u'', u'fp': u''}","{u'hr': u'', u'fp': u''}","{u'hr': {u'job': [{u'as_total_hours': u'614', ...","{u'hr': u'', u'fp': {u'job': {u'as_to': u'04/2...","{u'hr': {u'job': [{u'as_to': u'09/2011', u'as_...","{u'hr': u'', u'fp': {u'job': [{u'as_total_hour...","{u'hr': u'', u'fp': u''}","{u'hr': {u'job': {u'as_total_hours': u'2.33', ...","{u'hr': u'', u'fp': u''}",...,"{u'hr': u'', u'fp': u''}","{u'hr': u'', u'fp': u''}","{u'hr': u'', u'fp': u''}","{u'hr': u'', u'fp': u''}","{u'hr': {u'job': {u'as_total_hours': u'10.00',...","{u'hr': u'', u'fp': u''}","{u'hr': u'', u'fp': u''}","{u'hr': u'', u'fp': u''}","{u'hr': u'', u'fp': u''}","{u'hr': {u'job': [{u'as_total_hours': u'28', u..."


In [94]:
print as_profiles_df.dev_job_categories_v2.value_counts()

{u'dev_job_categories_v': {u'name': u'Web Development', u'groups': {u'group': {u'uid': u'531770282580668418', u'name': u'Web, Mobile & Software Dev'}}}}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [93]:
print as_profiles_df.dev_ac_agencies.value_counts()

                                                                                                                                                                                                                                                                                                                                                                                                                          7637
{u'dev_ac_agency': {u'ag_name': u'CSSA Info Media Pvt. Ltd.', u'ciphertext': u'~01c1dd1cf7e3e90d97', u'ag_logo': u'https://odesk-prod-portraits.s3.amazonaws.com/Companies:229385:CompanyLogoURL?AWSAccessKeyId=1XVAX3FNQZAFC9GJCFR2&Expires=2147483647&Signature=0EQPQKYiNLgmRZkmRWrM6zr1H2M%3D', u'ag_total_hours': u'16585.8333333333', u'ag_tot_feedback': u'234', u'ag_adj_score': u'4.5638057423'}}                    9
{u'dev_ac_agency': {u'ag_name': u'REMBROS CORPORATION DBA ITECHART GR', u'ciphertext': u'~01a17a64d4d5f499d5', u'ag_logo': u'https://www.odesk.com/att/~~fycV8WtafKa1GKaiX

In [ ]:
print as_profiles_df.dev_ui_profile_access.value_counts()

In [ ]:
print as_profiles_df.education[0:5]

In [ ]:
as_profiles_df[(as_profiles_df['dev_is_affiliated'] <> 0) & (as_profiles_df['dev_ac_agencies'] == '')][['dev_is_affiliated', 'dev_ac_agencies']]

In [ ]:
print as_profiles_df.dev_ac_agencies.value_counts()

In [ ]:
print as_profiles_lod[0]

In [ ]:
def json_prep_pandas(in_file):
    # read the entire file into a python array
    with open(in_file, 'rb') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)

    # each element of 'data' is an individual JSON object.
    # i want to convert it into an *array* of JSON objects
    # which, in and of itself, is one large JSON object
    # basically... add square brackets to the beginning
    # and end, and have all the individual business JSON objects
    # separated by a comma
    data_json_str = "[" + ','.join(data) + "]"

    data_list_of_dicts = json.loads(data_json_str)
    
#     now, load it into pandas
    out_df = pd.read_json(data_json_str)
    return out_df

In [ ]:
df = json_prep_pandas('../../data/detailed_profiles_da_0_8715.txt')

In [ ]:
df.info()

In [ ]:
df.assignments.head(10)

In [ ]:
for char in df.assignments.iloc[[0]]:
    print char

In [ ]:
for x in range(20):
    print x
    jobs = df.assignments[x]
    for k1, v1 in jobs.iteritems():        
        print 'k1 = ', k1
        if isinstance(v1, basestring):
            print 'basestring'
            pass
        elif isinstance(v1, dict):
            print 'dict'
            for k2, v2 in v1.iteritems():
                print '  k2 = ', k2
                if isinstance(v2, basestring):
                    print '  basestring'
                    pass
                elif isinstance(v2, dict):
                    print '  dict'
                    for k3, v3 in v2.iteritems():
                        print '    k3 = ' + k3
                elif isinstance(v2, list):
                    print '  list'
                    for index2, item in enumerate(v2):
                        print '    item', index2
                        if isinstance(item, dict):
                            for k4, v4 in item.iteritems():
                                print '    k4 = ', k4
        elif isinstance(v1, list):
            print 'list'
            for item in v1:
                print item
    print
    

In [ ]:
def triage(dict_, level=0):
    if not(isinstance(dict_, float) or isinstance(dict_, basestring)):
        print type(dict_)
        for key, val in dict_.iteritems():
            print ' '*level*2, key
            if isinstance(val, dict):
                triage(val, level=level+1)
            elif isinstance(val, list):
                for index, item in enumerate(val): 
                    print ' '*level*2, 'item', index
                    triage(item, level=level+1)
            elif isinstance(val, basestring):
                print ' '*(level+1)*2, '=', val

for x in range(1000,1020):
    print x
    jobs = df.dev_job_categories_v2[x]
    triage(jobs)

In [ ]:
for x in range(1000,1020):
    print x
    jobs = df.education[x]
    triage(jobs)

In [ ]:
q = 3.4
print isinstance(q, float)

In [ ]:
df.info()

In [ ]:
print as_profiles_lod[0]['ciphertext']

In [ ]:
print as_profiles_lod[0]['dev_recno_ciphertext']

In [ ]:
if profile['assignments']['hr'] = 

In [ ]:
print as_profiles_lod[0]['assignments']['hr']

In [ ]:
print type(as_profiles_lod[0]['assignments']['hr'])

In [ ]:
print type(as_profiles_lod[0]['assignments']['fp'])


In [ ]:
print as_profiles_lod[0]['assignments']['fp']

In [ ]:
print as_profiles_lod[1]['assignments']

In [ ]:
print type(as_profiles_lod[2]['assignments']['hr'])

In [ ]:
print as_profiles_lod[3]['assignments']['hr']['job'][0]

In [ ]:
print as_profiles_lod[3]['assignments']['hr']['job'][1]

In [ ]:
q = {'dog': 'day', 'cat': '~yay'}

In [ ]:
def possible_key(dict):
    key_flag = False
    possible_key = ''.join([dict[key] if '~' in dict[key] else '' for key in dict.keys()])
    if possible_key == '': 
        key_flag = True
        possible_key = None
    return key_flag, possible_key

In [ ]:
for index, dict_item in enumerate(as_profiles_lod):
    hr_assignments = dict_item['assignments']['hr']
    if isinstance(hr_assignments, basestring):
        if '~' in hr_assignments:
            return hr_assignments
    elif isinstance(hr_assignments, dict):
#         print "hr_assignments = dict"
        if 'job' in hr_assignments:
#             print 'job'
            hr_assignments_jobs = hr_assignments['job']
            if isinstance(hr_assignments_jobs, list):
#                 print 'is list'
                for job in hr_assignments_jobs:
                    if 'as_ciphertext_opening_recno' in job:
                        print job['as_ciphertext_opening_recno']
                    else:
                        poss_key, ciphertext = possible_key(dict)
                        if poss_key: return ciphertext
            elif isinstance(hr_assignments_jobs, dict):
#                 print 'is dict'
                if 'as_ciphertext_opening_recno' in hr_assignments_jobs:
                    print hr_assignments_jobs['as_ciphertext_opening_recno']
        elif 'assignment' in hr_assignments:
#             print 'assignment'
            hr_assignments_jobs = hr_assignments['assignment']
            if isinstance(hr_assignments_jobs, list):
                for job in hr_assignments_jobs:
                    if 'as_ciphertext_opening_recno' in job:
                        print job['as_ciphertext_opening_recno']
        else:
            print 'neither key'
            for job in hr_assignments:
                print type(job)
    elif isinstance(hr_assignments, list):
        print "hr_assignments = list"
        for job in hr_assignments:
            print job

#     print '---------'
    if index > 500: break
